# Numerical Langevin
### By Carlos A.C.C. Perello

## 0. Preliminaries
The only preliminaries are importing the relevant packages, i.e. the `LinearAlgebra.jl` package.

In [3]:
using LinearAlgebra

## 1. General computation of the derivative of the Brenier map

We are interested in transporting the following distributions:

$$
\mathcal{N}(m_1,\Sigma_1)\to\mathcal{N}(m_2, \Sigma_2)
$$

Where $\mathcal{N}(\mathbf{\mu}, \Sigma)$ is a multivariate normal with mean $\mathbf{\mu}\in \mathbb{R}^n$ and covariance matrix $\Sigma\in\mathbb{R}^{n\times n}$.

As we are interested in transporting a Gaussian distribution into another Gaussian distribution, the problem of computing the Brenier map (the unique map which is the gradient of a convex function, denoted as $T$) between these 2 distributions simplifies to computing the map [1]:

$$
T:x\mapsto m_2 + M(x-m_1) \quad \text{with } M = \Sigma_1^{-1/2}(\Sigma_1^{1/2}\Sigma_2\Sigma_1^{1/2})^{1/2}\Sigma_1^{-1/2}
$$

Therefore, the derivative of the Brenier map, $DT = M$, which is easy to compute and done below. We also want to compute the singular values of $DT$ as they can be used to bound its operator norm.

In [38]:
function DT(Σ₁, Σ₂)
    #=
    Returns a tuple consisting of 
    the derivative of the Bernier map in the first entry
    and a vector containing the singular vals
    of the derivative of the Bernier map in the second entry.
    =#
    ret = inv(√(Σ₁))*√(√(Σ₁)*Σ₂*√(Σ₁))*inv(√(Σ₁))
    ret
end

DT (generic function with 1 method)

#### Test

We generate 2 random pos. def matrices, which we will use for testing throughout the code.

In [39]:
n = 2 # Dimension of multivariate Gaussian
X₁ = rand(n, n)
X₂ = rand(n, n) # Generate 2 random n×n matrices
C₁ = X₁'*X₁
C₂ = X₂'*X₂ # Use the random matrices to generate 2 random 
# n×n pos. def. matrices, as covariance matrices are pos. def.
print(C₁)
print("\n")
print(C₂)

[0.6475510046028452 0.5636624464938351; 0.5636624464938351 0.9651296479505218]
[0.2157721546008205 0.26062974713696785; 0.26062974713696785 0.5597180591963227]

In [40]:
DT(C₁, C₂)

2×2 Matrix{Float64}:
 0.55577    0.0242436
 0.0242436  0.747252

## 2. Time-evolving computation the derivatives for the Brenier & Langevin maps

### 2.1 Computing $B_t$



We want to numerically compute the Langevin map between two Gaussians. We note that in this case, we are transporting:
$$
\mu = \mathcal{N}(\mu_1,\, \Sigma_1) \to \nu = \mathcal{N}(\mu_2,\, \Sigma_1 + \Sigma_2)
$$

To do so, we first need to introduce $P_t^A(f) $ and $B_t$. We define $P_t^A(f)$ as [2]:
$$
P_t^A(f) = \int_{\mathbb{R^n}}f\left(\exp(-tA)x + \sqrt{\text{Id}-\exp(-2tA)} y\right)d\mu(y)
$$

In the case of transport between 2 gaussians, $B_t$ is then defined as [2]:
$$
P_t^A\left(c_o\exp\left(-\frac{1}{2}y^TBy\right)\right)(x) = c_t\exp\left(-\frac{1}{2}x^T B_t x\right)
$$
And $B_t$ can be computed by explicitly computing and rearranging the LHS of the equation above [2].
We introduce the notation $A=\Sigma_1$, $B=\Sigma_2, C = e^{-tA}, D=\sqrt{I-e^{-2tA}}$ and $W = \sqrt{D^TBD + A}$ to obtain that $B_t$ is:
$$
B_t = C^TBC - C^TBDW^{-1}W^{-T}D^TB^TC
$$
and $c_t$ is given by:
$$
c_t = \frac{cc_0\sqrt{\pi}}{2\left|\det(W)\right|}
$$

We can think of $B_t$ as the "intermediate transport" between the two covariances; take $\mu_1 = \mu_2 = 0$ for simplicity:
- $B_0 = \Sigma_2$
- $B_\infty = 0$
- The covariance of an "intermediate Gaussian", i.e. if we truncate transport $\mathcal{N}(0,\Sigma_1) \to \mathcal{N}(0, \Sigma_1 + \Sigma_2)$ at time $t$ our resulting Gaussian will be $\mathcal{N}(0, \Sigma_1 + B_t)$


We can implement $B_t$ and $c_t$ as functions, done below.

In [12]:
function Bₜ(Σ₁, Σ₂, t)
    C = exp(-t.*Σ₁)
    D = √(I-exp(-(2*t).*Σ₁))
    W = √(D'*Σ₂*D + Σ₁)
    prod = C'*Σ₂*D*inv(W)
    ret = C'*Σ₂*C - prod*prod'
    ret
end

function cₜ(c, c₀, Σ₁, Σ₂, t)
    D = √(I-exp(-(2*t).*Σ₁))
    W = √(D'*Σ₂*D + Σ₁)
    ret = (c*c₀*(√π))/(2*abs(det(W)))
    ret
end

cₜ (generic function with 1 method)

#### 2.1.1 Test

$B_t$ should satisfy $B_0 = B = \Sigma_2$ [2] which we can use as a sanity check. Additionally, tending $t\to\infty$ in the expression for $P^t_A(f)$ with our choice of $f$ yields that $B_\infty = \lim\limits_{t\to\infty}B_t = \log(\text{Id}) = \mathbf{0}_{2\times 2}$. To test this, we reuse the random positive definite matrices generated above:

In [33]:
Bₜ(C₁, C₂, 0) == C₂ && Bₜ(C₁, C₂, 10000000) == zeros(2, 2)

true

### 2.2 Derivative of Brenier map

Now let $\nu_t$ be the resulting measure transforming $\mu$ up to time step $t$. In this case, $\nu_0 = \nu$ and $\nu_\infty = \mu$. Let the Brenier map transporting $\mu\to\nu_t$ be $T_t$.

Then [2] tells us that the derivative of $T_t$ is given by:
$$
DT_t = \Sigma_1^{1/2}(\Sigma_1^{1/2}(\Sigma_1+B_t)\Sigma_1^{1/2})^{-1/2}\Sigma_1^{1/2}
$$
Which is implemented below.

In [42]:
function DTₜ(Σ₁, Σ₂, t)
    #= Map between μ and νₜ, where:
    μ = N(0, Σ₁)
    ν = N(0, Σ₁ + Σ₂)
    νₜ = N(0, Σ₁ + Bₜ) ⟹ ν₀ = ν, ν∞ = μ
    =#
    Bt = Bₜ(Σ₁, Σ₂, t)
    rootΣ₁ = √(Σ₁)
    ret = inv(rootΣ₁)*√(rootΣ₁*(Σ₁ + Bt)*rootΣ₁)*inv(rootΣ₁)
    ret
end

DTₜ (generic function with 1 method)

#### 2.2.1 Test

To test, we check that $DT_0$ indeed maps $\mu\to\nu$ and $DT_\infty$ indeed maps $\mu \to \mu$:

In [47]:
DTₜ(C₁, C₂, 0) ≈ DT(C₁, C₁ + C₂) && DTₜ(C₁, C₂, 100000) ≈ I

true

### 2.3 Derivative of Langevin map

Let $S_t(x)$ be the Langevin map at point $x$ at time $t$.

Then, once we have $B_t$, the Langevin map is given by solving the following PDE:
$$
\frac{\partial S_t(x)}{\partial t} = B_t S_t(x)
$$

Fixing $x$, this becomes:

$$
\frac{d S_t(x)}{dt} = B_t S_t(x)
$$

Which we can solve using a basic ODE solver, such as the Forward Euler method [3].

#### 2.3.1 Langevin map via Forward Euler


We now fix $x$ and solve for $S_t(x)$. As we fix $x$, it will be easier to write $S_t(x) = S_x(t)$. We first discretise the time dimension into $n$ equal sized time steps between 0 and $T$.

In [9]:
T = 10
n = 10_000
t = range(0, T, length=n)
h = step(t)

0.001000100010001

Now, we use the Forward Euler method, given by the following equation, to solve for $S_x(t)$:
$$
S_x(0) = x,\, \text{ as } t_1 = 0\\
S_x(t_{k+1}) = \left(\text{Id} + hB_{t_k}\right)S_x(t_{k}),\, 2\leq k\leq n
$$
This is implemented below as a function that takes in $x$ as its initial condition and outputs the time evolution of the Langevin map for the discrete time steps betwen $t=0$ and $t=T$.

In [10]:
function S(Σ₁, Σ₂, n, T, x, full=false)
    # full=True returns array of time evolution, if false just returns value at end T
    t = range(0, T, length=n)
    h = step(t)
    if full
        Sₜx = zeros(2, n)
        Sₜx[:, 1] = x
        for k = 1:n-1
            Sₜx[:, k+1] = (I + h.*Bₜ(Σ₁, Σ₂, t[k]))*Sₜx[:, k]
        end
    else
        Sₜx = x
        for k = 1:n-1
            Sₜx = (I + h.*Bₜ(Σ₁, Σ₂, t[k]))*Sₜx
        end
    end
    Sₜx
end

LT_map (generic function with 2 methods)

#### 2.3.2 Derivative of Langevin map using Forward Euler

We can also solve for the derivative, $DS_x(t)$, by using solving the following differential equation:
$$
\frac{dDS_x(t)}{dt} = B_tDS_t(x);\; DS_x(0) = \text{Id}
$$

Note that as $DS_x(0)$ is independent of $x \, \forall x\in \mathbb{R^2}$, $DS_x(t)$ will be independent of $x$ too, meaning we can simply write $DS_x(t) = DS(t)$.

We can do this by extending Forward Euler to solve matrix ODEs; we have:
$$
DS(0) = \text{Id},\, \text{ as } t_1 = 0\\
DS(t_{k+1}) = \left(\text{Id} + hB_{t_k}\right)DS(t_{k}),\, 2\leq k\leq n
$$

This is implemented below. Note that we define `gen_step` to generate a nonuniformly spaced discretization of time; this is done to replicate the numerical methods implemented in [2] which yielded accurate results.

In [180]:
function gen_step(T, tol=0.001)
    if T <= 0.1
        ret = 0:0.02*tol:T
    elseif T <= 0.5 && T > 0.1
        ret = [0:0.02*tol:0.1; 0.1:0.05*tol:T]
    elseif T <= 1 && T > 0.5
        ret = [0:0.02*tol:0.1; 0.1:0.05*tol:0.5; 0.5:0.2*tol:T]
    elseif T <= 3 && T > 1
        ret = [0:0.02*tol:0.1; 0.1:0.05*tol:0.5; 0.5:0.2*tol:1; 1:0.5*tol:T]
    elseif T <= 5 && T > 3
        ret = [0:0.02*tol:0.1; 0.1:0.05*tol:0.5; 0.5:0.2*tol:1; 1:0.5*tol:3; 3:2*tol:T]
    else
        ret = [0:0.02*tol:0.1; 0.1:0.05*tol:0.5; 0.5:0.2*tol:1; 1:0.5*tol:3; 3:2*tol:5; 5:5*tol:T]
    end
end

gen_step (generic function with 2 methods)

In [237]:
#=
function DS_uniform(Σ₁, Σ₂, h, T)
    t = 0:h:T
    n = length(t)
    S∞ = [1 0; 0 1]
    for k = 1:n-1
        S∞ = (I + h.*Bₜ(Σ₁, Σ₂, t[k]))*S∞
    end
    S∞
end
=#
function DS(Σ₁, Σ₂, T, tol=0.0001, conv=0)
    #= Numerically compute the derivative of the Langevin map
    between μ = N(0, Σ₁) -> ν = N(0, Σ₁ + Σ₂) (at time ∞)
    Parameters:
    - Σ₁, Σ₂: Covariance matrices.
    - T: Last time when the Fwd. Euler method is applied.
    - tol: Order of magnitude to which we want to approximate DS to the actual value. 
           Default 0.0001 ⟹ using [2]'s inhomogeneous time spacing.
    - conv: If the norm of the difference of subsequent iterations is below this value,
            we stop. Default 0 ⟹ we finish until end of iteration.
    =#
    realtol = tol*10
    t = gen_step(T, realtol)
    n = length(t)
    S∞ = [1 0; 0 1]
    hold = [1 0; 0 1]
    for k = 1:n-1
        hold = S∞
        h = t[k+1] - t[k]
        S∞ = (I + h.*Bₜ(Σ₁, Σ₂, t[k]))*S∞
        if norm(S∞-hold)<conv 
            return S∞
        end
    end
    S∞
end


DS (generic function with 3 methods)

#### 2.3.3 Test

We test our code by taking $\Sigma_1$ to be diagonal so it commutes with $\Sigma_2$. Once again, according to [2], if $\Sigma_1$ and $\Sigma_2$ commute, $\lim\limits_{t\to\infty}DS(t) = DT_0$. (more on why)

In [239]:
D₁ = Diagonal([2, 9])
D₂ = Diagonal([10, 6])

2×2 Diagonal{Int64, Vector{Int64}}:
 10  ⋅
  ⋅  6

In [240]:
0.000001 - 1e-6

0.0

In [244]:
T = 30 # take T to be large to be an ok approximation of ∞ but small so we can converge in "fast-ish" time.
DScomm = DS(D₁, D₂, T, 1e-5) # the more we decrease h the better the approx, but the longer the wait

2×2 Matrix{Float64}:
 2.44952  0.0
 0.0      1.291

In [245]:
norm(DTₜ(D₁, D₂, 0) - DScomm)

3.102608255820665e-5

### Brainstorm

One first guess for the Langevin map was:
$$
G_1(t) = \sqrt{(\Sigma_1+B_t)\Sigma_1^{-1}}
$$
Which works when $\Sigma_1$ and $\Sigma_2$ commute, as in that case $G_1$ is the Brenier map. However, this is not the case for general $\Sigma_1$ and $\Sigma_2$ as verified below.

In [152]:
function G₁(Σ₁, Σ₂, t)
    ret = √((Σ₁+Bₜ(Σ₁, Σ₂, t))*inv(Σ₁))
    ret
end

norm(G(C₁, C₂, 30) - DS(C₁, C₂, 30)) ≈ 0 # not close to 0

false

Another guess would be to do the following:
- Let $\Sigma_1 = P D P^T $ and $\Sigma_1 + B_t = P_t D_t P_t^T$.


Then the guess is:
$$
G_2(t) = P_t(D_t D^{-1})^{1/2}P^T
$$

  

In [102]:
function G₂(Σ₁, Σ₂, t)
    Bt = Bₜ(Σ₁, Σ₂, t) # Compute Bₜ
    evals, P = eigen(Σ₁) # Decompose Σ₁
    evalsₜ, Pₜ = eigen(Σ₁+Bt) # Decompose Σ₁ + Bₜ
    D = Diagonal(evals) # Make Diagonal matrix of evals of Σ₁
    Dₜ = Diagonal(evalsₜ) # Make Diagonal matrix of evals of Σ₁ + Bₜ
    ret = Pₜ*√(Dₜ*inv(D))*P' # return the guess
    ret
end

G₂ (generic function with 1 method)

This does not seem to yield a good approximation either. However, this is observed:

In [171]:
norm(G₂(C₁, C₂, 30) - DS(C₁, C₂, 0))

2.480065319449395e-7

In [168]:
DTₜ(D₁, D₂, 0)

2×2 Diagonal{Float64, Vector{Float64}}:
 2.44949   ⋅ 
  ⋅       1.29099

In [165]:
G₂(D₁, D₂, 30)

2×2 Matrix{Float64}:
 1.0  0.0
 0.0  1.0

In [166]:
DS(D₁, D₂, 30)

2×2 Matrix{Float64}:
 2.44979  0.0
 0.0      1.29108

In [163]:
norm(G₂(D₁, D₂, 30) - DS(D₁, D₂, 30))

1.4787203749737685

## 3. Properties of DS and DT

### 3.1 Non-commuting $\Sigma_1$ and $\Sigma_2$

Consider the case where we transport $$\mu = \mathcal{N}(0, \Sigma_1) \to \nu = \mathcal{N}(0, \Sigma_1 + \Sigma_2)$$

We now explore the case when $\Sigma_1$ and $\Sigma_2$ do not commute. We first ask the question: Can we find noncommuting $\Sigma_1$ and $\Sigma_2$ such that $DT_0 = DS_\infty$? We first implement a function to generate a random noncommuting pair of matrices.

In [287]:
function gen_noncommuting_pair()
    A₁ = randn(2, 2)
    A₂ = randn(2, 2)
    S₁ = A₁'*A₁
    S₂ = A₂'*A₂
    while S₁*S₂ == S₂*S₁
        A₁ = randn(2, 2)
        A₂ = randn(2, 2)
        S₁ = A₁'*A₁
        S₂ = A₂'*A₂
    end
    S₁, S₂
end

gen_noncommuting_pair (generic function with 1 method)

Now, we generate random pos. def. symmetric matrices $S_1$ and $S_2$ and consider the transport:

$$\mu = \mathcal{N}(0, S_1) \to \nu = \mathcal{N}(0, S_1 + S_2)$$

The code below computes $DS_\infty$ withing tolerance $\varepsilon_1$ for this transport problem and checks if this is within tolerance $\varepsilon_2$ of the Brenier map , $DT_0$, for this transport problem. 
If not, we resample $S_1$ and $S_2$ and we try until we find 2 matrices $S_1,S_2$ s.t $DS_\infty$ and $DT_0$ are close.

In [298]:
function identical_transports(T, ε₁, ε₂)
    S₁, S₂ = gen_noncommuting_pair()
    diff = norm(DS(S₁, S₂, T, ε₁) - DTₜ(S₁, S₂, 0))
    while diff >= ε₂
        S₁, S₂ = gen_noncommuting_pair()
        diff = norm(DS(S₁, S₂, T, ε₁) - DTₜ(S₁, S₂, 0))
    end
    S₁, S₂
end

identical_transports (generic function with 1 method)

In [309]:
A₁, B₁ = identical_transports(30, 1e-3, 1e-4)

([0.9584804037413681 -2.6721423562912765; -2.6721423562912765 10.708243929398195], [0.03205788111617298 -0.13627317075623516; -0.13627317075623516 0.6174151823306084])

The following matrices satisfy this condition:

In [310]:
println(A)
println(B)
println(DS(A,B,30,1e-3))
println(DTₜ(A,B,0))
println(norm(DS(A,B,30,1e-3) - DTₜ(A,B,0)))

[6.307103734532885 0.6662019609743299; 0.6662019609743299 2.5708796185661447]
[0.17472400613533526 -0.0904076437370299; -0.0904076437370299 0.04985633494910014]
[1.0150390415093176 -0.01210256844715929; -0.012181650127149184 1.0126249778124832]
[1.0150070023883802 -0.012107929147744098; -0.012107929147744112 1.0126141231941128]
8.128867033979074e-5


Now we check if we can have the Lipschitz constant of $DT_0$ can be larger than the Lipschitz constant of $DS_\infty$. The code below samples random symmetric pos. def. matrices until this condition is satisfied:

In [311]:
function lip_DT_larger_DS(T, ε₁, ε₂)
    S₁, S₂ = gen_noncommuting_pair()
    DS∞ = DS(S₁, S₂, T, ε₁)
    DT₀ = DTₜ(S₁, S₂, 0)
    lip_DS∞ = svdvals(DS∞)[1]
    lip_DT₀ = svdvals(DT₀)[1]
    while norm(DS∞ - DT₀) < ε₂ || lip_DS∞ >= lip_DT₀
        S₁, S₂ = gen_noncommuting_pair()
        DS∞ = DS(S₁, S₂, T, ε₁)
        DT₀ = DTₜ(S₁, S₂, 0)
        lip_DS∞ = svdvals(DS∞)[1]
        lip_DT₀ = svdvals(DT₀)[1]
    end
    S₁, S₂
end

lip_DT_larger_DS (generic function with 1 method)

Here we can check that this is indeed true:

In [314]:
A₂, B₂ = lip_DT_larger_DS(30, 1e-3, 1e-4)
svdvals(DTₜ(A₂, B₂, 0))[1]  > svdvals(DS(A₂, B₂, 30, 1e-3))[1] 

true

We now do the same, but we check the opposite condition; we check if the Lipschitz constant of $DS_\infty$ can be larger than that of $DT_0$, which is what the code below does.

In [321]:
function lip_DT_smaller_DS(T, ε₁, ε₂)
    S₁, S₂ = gen_noncommuting_pair()
    DS∞ = DS(S₁, S₂, T, ε₁)
    DT₀ = DTₜ(S₁, S₂, 0)
    lip_DS∞ = svdvals(DS∞)[1]
    lip_DT₀ = svdvals(DT₀)[1]
    while norm(DS∞ - DT₀) < ε₂ || lip_DS∞ <= lip_DT₀
        S₁, S₂ = gen_noncommuting_pair()
        DS∞ = DS(S₁, S₂, T, ε₁)
        DT₀ = DTₜ(S₁, S₂, 0)
        lip_DS∞ = svdvals(DS∞)[1]
        lip_DT₀ = svdvals(DT₀)[1]
    end
    S₁, S₂
end

lip_DT_smaller_DS (generic function with 1 method)

Which we can also check:

In [322]:
A₃, B₃ = lip_DT_smaller_DS(30, 1e-3, 1e-4)
svdvals(DTₜ(A₃, B₃, 0))[1] < svdvals(DS(A₃, B₃, 30, 1e-3))[1] 

true

Lastly, we check if they can be equal, i.e. if they can be within a distance of $\varepsilon_3$:

In [323]:
function lip_DT_approx_DS(T, ε₁, ε₂, ε₃)
    S₁, S₂ = gen_noncommuting_pair()
    DS∞ = DS(S₁, S₂, T, ε₁)
    DT₀ = DTₜ(S₁, S₂, 0)
    lip_DS∞ = svdvals(DS∞)[1]
    lip_DT₀ = svdvals(DT₀)[1]
    while norm(DS∞ - DT₀) < ε₂ || norm(lip_DS∞ - lip_DT₀) > ε₃
        S₁, S₂ = gen_noncommuting_pair()
        DS∞ = DS(S₁, S₂, T, ε₁)
        DT₀ = DTₜ(S₁, S₂, 0)
        lip_DS∞ = svdvals(DS∞)[1]
        lip_DT₀ = svdvals(DT₀)[1]
    end
    S₁, S₂
end

lip_DT_approx_DS (generic function with 1 method)

In [333]:
A₄, B₄ =  lip_DT_approx_DS(30, 1e-3, 1e-4, 1e-4)
norm(svdvals(DTₜ(A₄, B₄, 0))[1] - svdvals(DS(A₄, B₄, 30, 1e-3))[1])

7.558539725338598e-5

# References
[1] - https://djalil.chafai.net/blog/2010/04/30/wasserstein-distance-between-two-gaussians/

[2] - Anastasiya Tanana (2020). Comparison of transport map generated by heat flow interpolation and the optimal transport Brenier map. Communications in Contemporary Mathematics, 23(06), 2050025.

[3] - https://github.com/Imperial-MATH50003/MATH50003NumericalAnalysis/blob/main/notes/MATH50003_numerical_analysis_lecture_notes.pdf

Check